# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset: CO2_Data

Import the necessary libraries and create your dataframe(s).

In [7]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('CO2_Data.csv')

# Create a copy for cleaning to keep the original data safe
df_clean = df.copy()

# Quick check to ensure the columns we identified (Name, year, co2, iso_code) are present
print("Initial Dataframe Shape:", df_clean.shape)
print("\nFirst 5 rows:")
print(df_clean[['Name', 'year', 'co2', 'iso_code']].head())

Initial Dataframe Shape: (43746, 80)

First 5 rows:
          Name  year  co2 iso_code
0  Afghanistan  1850  NaN      AFG
1  Afghanistan  1851  NaN      AFG
2  Afghanistan  1852  NaN      AFG
3  Afghanistan  1853  NaN      AFG
4  Afghanistan  1854  NaN      AFG


## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [8]:
# Test for Missing Data
# Calculating the total number of missing values for each column
missing_values = df_clean.isnull().sum()
missing_percentage = (df_clean.isnull().sum() / len(df_clean)) * 100

# Consolidating the report to see the impact
missing_report = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
}).sort_values(by='Missing Values', ascending=False)

print("\nMissing Data Report (Top 10):")
print(missing_report.head(10))

# Handling Missing Data

# Rows with a missing CO2 value provide no information for analysis so I'll drop them.
initial_rows = len(df_clean)
df_clean = df_clean.dropna(subset=['co2'])
print(f"\nAction: Dropped {initial_rows - len(df_clean)} rows where 'co2' was missing.")

# While these aren't 'countries', they are useful for high-level summaries.
# I won't drop them yet, but I'll fill the NaN with 'Aggregate' to make it clear.
df_clean['iso_code'] = df_clean['iso_code'].fillna('AGG')
print("Action: Labeled missing 'iso_code' as 'AGG' for regional aggregates.")

# Dropping all rows with missing GDP would wipe out most of our historical CO2 data (pre-1960).
# Instead of dropping, I will leave them as NaN for now and only filter them when I specifically perform "per capita" or "CO2 vs GDP" calculations.
print("Decision: Retained rows with missing GDP/Population to preserve historical CO2 trends.")

# Final Validation
print("\nPost-Cleaning Missing Count for key columns:")
print(df_clean[['Name', 'year', 'co2', 'iso_code']].isnull().sum())

# Export the partially cleaned data for the next step
df_clean.to_csv('cleaned_co2_data.csv', index=False)


Missing Data Report (Top 10):
                                   Missing Values  Percentage
share_global_other_co2                      41638   95.181274
share_global_cumulative_other_co2           41638   95.181274
other_co2_per_capita                        41282   94.367485
cumulative_other_co2                        40844   93.366251
other_industry_co2                          40844   93.366251
consumption_co2_per_gdp                     39302   89.841357
consumption_co2_per_capita                  39244   89.708773
trade_co2_share                             39211   89.633338
trade_co2                                   39211   89.633338
consumption_co2                             38880   88.876697

Action: Dropped 16687 rows where 'co2' was missing.
Action: Labeled missing 'iso_code' as 'AGG' for regional aggregates.
Decision: Retained rows with missing GDP/Population to preserve historical CO2 trends.

Post-Cleaning Missing Count for key columns:
Name        0
year        0
co2 

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [9]:
# Load our cleaned data from the previous step
df_clean = pd.read_csv('cleaned_co2_data.csv')

#  Identify Statistical Outliers using IQR
# Values 1.5 times the IQR above the 75th percentile are statistically outliers.

Q1 = df_clean['co2'].quantile(0.25)
Q3 = df_clean['co2'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_df = df_clean[df_clean['co2'] > upper_bound]

print(f"Statistical Upper Bound: {upper_bound:.2f}")
print(f"Number of statistical outliers detected: {len(outliers_df)}")

# Analyze the Outliers
print("\nSample of detected outliers:")
print(outliers_df[['Name', 'year', 'co2', 'iso_code']].tail(10))

# 1. Regional Aggregates (iso_code == 'AGG') like 'World' or 'Asia'.
# 2. Large industrial nations (USA, China, etc.) in recent decades.
# We MUST NOT delete the countries, but the 'AGG' rows will double-count data if we mix them with individual countries.

# Handling the Outliers

# This is the safest way to handle high-value "outliers" without losing data.
df_countries = df_clean[df_clean['iso_code'] != 'AGG'].copy()
df_aggregates = df_clean[df_clean['iso_code'] == 'AGG'].copy()


impossible_outliers = df_countries[df_countries['co2'] < 0]
if not impossible_outliers.empty:
    df_countries = df_countries[df_countries['co2'] >= 0]
    print(f"\nAction: Removed {len(impossible_outliers)} rows with negative CO2 values.")
else:
    print("\nNo negative CO2 values detected.")

# Final Review
print(f"\nFinal Country-only Dataframe Shape: {df_countries.shape}")
print(f"Max CO2 in Countries (Post-split): {df_countries['co2'].max():.2f}")

# Save the country-only data for focused analysis
df_countries.to_csv('countries_co2_no_outliers.csv', index=False)

Statistical Upper Bound: 149.32
Number of statistical outliers detected: 4959

Sample of detected outliers:
                         Name  year      co2 iso_code
27049  International shipping  2014  621.561      AGG
27050  International shipping  2015  638.760      AGG
27051  International shipping  2016  632.392      AGG
27052  International shipping  2017  645.677      AGG
27053  International shipping  2018  641.295      AGG
27054  International shipping  2019  627.170      AGG
27055  International shipping  2020  583.679      AGG
27056  International shipping  2021  599.905      AGG
27057  International shipping  2022  595.232      AGG
27058  International shipping  2023  593.158      AGG

No negative CO2 values detected.

Final Country-only Dataframe Shape: (22248, 80)
Max CO2 in Countries (Post-split): 11902.50


## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [10]:
# Load the country-only data we saved in the last step
df = pd.read_csv('countries_co2_no_outliers.csv')

# Identifying and Removing Redundant Columns 

# Drop columns that are entirely empty
initial_cols = df.shape[1]
df.dropna(axis=1, how='all', inplace=True)
dropped_cols = initial_cols - df.shape[1]

# For this script, we'll stick to dropping columns with > 95% missing values.
threshold = len(df) * 0.05
df = df.dropna(thresh=threshold, axis=1)

print(f"Action: Dropped {initial_cols - df.shape[1]} columns that were empty or mostly null.")

# Identifying and Removing Duplicate Rows
# They artificially inflate totals and skew averages.
duplicates_count = df.duplicated(subset=['Name', 'year']).sum()

if duplicates_count > 0:
    df.drop_duplicates(subset=['Name', 'year'], keep='first', inplace=True)
    print(f"Action: Removed {duplicates_count} duplicate records found for Name + Year.")
else:
    print("Action: No duplicate rows detected.")

# Filtering Unnecessary Timeframes
# Most economic comparisons (GDP) are only valid from 1950 onwards.
# Let's filter for data from 1900 onwards to keep historical context but remove extreme outliers.
df = df[df['year'] >= 1900]
print(f"Action: Filtered dataset to start from year 1900. Current rows: {len(df)}")

# Removing 'Other' or 'Unknown' Entities 
unnecessary_names = ['Kuwaiti Oil Fires', 'International transport']
df = df[~df['Name'].isin(unnecessary_names)]

# Final Structure Check
print("\nFinal Remaining Columns for Analysis:")
print(df.columns.tolist())

# Save the lean dataset
df.to_csv('final_clean_co2_data.csv', index=False)

Action: Dropped 0 columns that were empty or mostly null.
Action: No duplicate rows detected.
Action: Filtered dataset to start from year 1900. Current rows: 19572

Final Remaining Columns for Analysis:
['Description', 'Name', 'year', 'iso_code', 'population', 'gdp', 'cement_co2', 'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct', 'co2_including_luc', 'co2_including_luc_growth_abs', 'co2_including_luc_growth_prct', 'co2_including_luc_per_capita', 'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy', 'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2', 'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita', 'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2', 'cumulative_co2_including_luc', 'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2', 'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita', 'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas

## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [11]:
# Load our filtered and lean dataset
df = pd.read_csv('final_clean_co2_data.csv')

# Standardizing Country Names
# We'll check for common variations and standardize them.
name_mapping = {
    'United States': 'USA',
    'United States of America': 'USA',
    'United Kingdom': 'UK',
    'Viet Nam': 'Vietnam',
    'Czechia': 'Czech Republic',
    'Russian Federation': 'Russia'
}

# We apply a mapping to ensure consistent naming for major entities
df['Name'] = df['Name'].replace(name_mapping)

# Also, strip any leading/trailing whitespace that might cause grouping errors
df['Name'] = df['Name'].str.strip()
print("Action: Standardized country names and stripped whitespace.")

# Logical Consistency Check
# We'll check for rows where the sum of parts is significantly greater than the total.

# We define the component columns
components = ['coal_co2', 'oil_co2', 'gas_co2', 'cement_co2', 'flaring_co2']

# Fill NaNs in components with 0 temporarily just for the calculation
temp_sum = df[components].fillna(0).sum(axis=1)

# Check for rows where the sum of components is more than 5% greater than the 'co2' total
logical_errors = df[temp_sum > (df['co2'] * 1.05)]

if not logical_errors.empty:
    print(f"Action: Found {len(logical_errors)} rows with logical sum inconsistencies.")
    # In these cases, it's often safer to re-calculate the total 'co2' 
    # as the sum of components to ensure internal consistency.
    df.loc[temp_sum > (df['co2'] * 1.05), 'co2'] = temp_sum
else:
    print("Action: No significant logical inconsistencies found in CO2 components.")

# Population and GDP Consistency
# If they are, they are likely data entry errors or placeholders.
pop_errors = df[df['population'] <= 0]
if not pop_errors.empty:
    df.loc[df['population'] <= 0, 'population'] = np.nan
    print(f"Action: Replaced {len(pop_errors)} zero/negative population values with NaN.")

# Chronological Consistency
# We'll ensure there are no duplicate years per country (already handled, but good to verify).
df = df.sort_values(by=['Name', 'year'])

# Data Type Consistency
numeric_cols = ['co2', 'population', 'gdp', 'year']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nFinal Consistency Check Complete.")
print(df[['Name', 'year', 'co2']].head())

# Save the fully cleaned and consistent dataset
df.to_csv('final_consistent_co2_data.csv', index=False)

Action: Standardized country names and stripped whitespace.
Action: Found 10 rows with logical sum inconsistencies.

Final Consistency Check Complete.
          Name  year    co2
0  Afghanistan  1949  0.015
1  Afghanistan  1950  0.084
2  Afghanistan  1951  0.092
3  Afghanistan  1952  0.092
4  Afghanistan  1953  0.106


## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?
Yes. 
Missing: Many years were missing GDP and population numbers.
Outliers: The "World" and "Asia" totals were so huge they made individual countries look tiny.
Unnecessary: There were empty columns we didn't need and rows for things like "International Transport" that aren't actual countries.
Inconsistent: Some countries were spelled in different ways, and a few totals didn't match the sum of their parts.

2. Did the process of cleaning your data give you new insights into your dataset?
Yes. I realized that the data mixes countries with entire regions. If I hadn't cleaned it, the "World" total would have been mistaken for a single country, which would have ruined my rankings and averages.

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?
Yes. Since big countries have massive emissions compared to small ones, my charts might look lopsided. I also need to be careful when comparing CO2 to population, because the population data usually stops a few years before the CO2 data does.